# recherche

### Import libraries

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = 100

import requests
from bs4 import BeautifulSoup
import re
from multiprocessing import Pool
import pandas as pd

import pandas as pd
import numpy as np
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

import validators

# -------------------------------------------------

# -------------------------------------------------

# -------------------------------------------------

In [2]:
export = pd.read_csv("data_sample/20150218230000.export.CSV", sep="\t", header = None)
mentions = pd.read_csv("data_sample/20150218230000.mentions.CSV", sep="\t", header=None)
gkg = pd.read_csv("data_sample/20150218230000.gkg.csv", sep="\t", header=None)
gkg_translation = pd.read_csv("data_sample/20150218230000.translation.gkg.csv", sep="\t", header=None)
mentions_translation = pd.read_csv("data_sample/20150218230000.translation.mentions.CSV", sep="\t", header=None)
export_translation = pd.read_csv("data_sample/20150218230000.translation.export.CSV", sep="\t", header = None)

d_export = {
0 : "GlobalEventID",
1 : "Day",
2 : "MonthYear",
3 : "Year",
4 : "FractionDate",
5 : "Actor1Code",
6 : "Actor1Name",
7 : "Actor1CountryCode",
8 : "Actor1KnownGroupCode",
9 : "Actor1EthnicCode",
10 : "Actor1Religion1Code",
11 : "Actor1Religion2Code",
12 : "Actor1Type1Code",
13 : "Actor1Type2Code",
14 : "Actor1Type3Code",
15 : "Actor2Code",
16 : "Actor2Name",
17 : "Actor2CountryCode",
18 : "Actor2KnownGroupCode",
19 : "Actor2EthnicCode",
20 : "Actor2Religion1Code",
21 : "Actor2Religion2Code",
22 : "Actor2Type1Code",
23 : "Actor1Type2Code",
24 : "Actor1Type3Code",
25 : "IsRootEvent",
26 : "EventCode",
27 : "EventBaseCode",
28 : "EventRootCode",
29 : "QuadClass",
30 : "GoldsteinScale",
31 : "NumMentions",
32 : "NumSources",
33 : "NumArticles",
34 : "AvgTone",
35 : "Actor1Geo_Type",
36 : "Actor1Geo_Fullname",
37 : "Actor1Geo_CountryCode",
38 : "Actor1Geo_ADM1Code",
39 : "Actor1Geo_ADM2Code",
40 : "Actor1Geo_Lat",
41 : "Actor1Geo_Long",
42 : "Actor1Geo_FeatureID",
43 : "Actor2Geo_Type",
44 : "Actor2Geo_Fullname",
45 : "Actor2Geo_CountryCode",
46 : "Actor2Geo_ADM1Code",
47 : "Actor2Geo_ADM2Code",
48 : "Actor2Geo_Lat",
49 : "Actor2Geo_Long",
50 : "Actor2Geo_FeatureID",
51 : "ActionGeo_Type",
52 : "ActionGeo_Fullname",
53 : "ActionGeo_CountryCode",
54 : "ActionGeo_ADM1Code",
55 : "ActionGeo_ADM2Code",
56 : "ActionGeo_Lat",
57 : "ActionGeo_Long",
58 : "ActionGeo_FeatureID",
59 : "DATEADDED",
60 : "SOURCEURL"}

d_mentions = {
0  : 'GlobalEventID',
1  : 'EventTimeDate',
2  : 'MentionTimeDate',
3  : 'MentionType',
4  : 'MentionSourceName',
5  : 'MentionIdentifier',
6  : 'SentenceID',
7  : 'Actor1CharOffset',
8  : 'Actor2CharOffset',
9  : 'ActionCharOffset',
10  : 'InRawText',
11  : 'Confidence',
12  : 'MentionDocLen',
13  : 'MentionDocTone',
14  : 'MentionDocTranslationInfo',
15  : 'Extras',
}


d_gkg = {
0  : 'GKGRECORDID',
1  : 'V2.1DATE',
2  : 'V2SOURCECOLLECTIONIDENTIFIER',
3  : 'V2SOURCECOMMONNAME',
4  : 'V2DOCUMENTIDENTIFIER',
5  : 'V1COUNTS',
6  : 'V2.1COUNTS',
7  : 'V1THEMES',
8  : 'V2ENHANCEDTHEMES',
9  : 'V1LOCATIONS',
10  : 'V2ENHANCEDLOCATIONS',
11  : 'V1PERSONS',
12  : 'V2ENHANCEDPERSONS',
13  : 'V1ORGANIZATIONS',
14  : 'V2ENHANCEDORGANIZATIONS',
15  : 'V1.5TONE',
16  : 'V2.1ENHANCEDDATES',
17  : 'V2GCAM',
18  : 'V2.1SHARINGIMAGE',
19  : 'V2.1RELATEDIMAGES',
20  : 'V2.1SOCIALIMAGEEMBEDS',
21  : 'V2.1SOCIALVIDEOEMBEDS',
22  : 'V2.1QUOTATIONS',
23  : 'V2.1ALLNAMES',
24  : 'V2.1AMOUNTS',
25  : 'V2.1TRANSLATIONINFO',
26  : 'V2EXTRASXML',
}


export.rename(columns=d_export, inplace=True)
mentions.rename(columns=d_mentions, inplace=True)
gkg.rename(columns=d_gkg, inplace=True)

gkg_translation.rename(columns=d_gkg, inplace=True)
mentions_translation.rename(columns=d_mentions, inplace=True)
export_translation.rename(columns=d_export, inplace=True)

# -------------------------------------------------

# -------------------------------------------------

# -------------------------------------------------

# Lecture ZIP - CSV

In [25]:
response = requests.get("http://data.gdeltproject.org/gdeltv2/masterfilelist.txt")
content = response.content.decode("utf-8") 
l = content.split('\n')[:200]

liste = list()

for i in l: liste.append(i.split(" ")[-1])

In [26]:
df = pd.DataFrame(liste, columns=['url'])

df['annee'] = df['url'].apply(lambda x : x.split("/")[-1].split(".")[0][:4])
df['mois'] = df['url'].apply(lambda x : x.split("/")[-1].split(".")[0][4:6])
df['jour'] = df['url'].apply(lambda x : x.split("/")[-1].split(".")[0][6:8])
df['heur'] = df['url'].apply(lambda x : x.split("/")[-1].split(".")[0][8:10])
df['minute'] = df['url'].apply(lambda x : x.split("/")[-1].split(".")[0][10:12])

df['type_csv'] = df['url'].apply(lambda x : x.lower().split(".csv")[0].split(".")[-1])

df

,url,annee,mois,jour,heur,minute,type_csv
0,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,00,export
1,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,00,mentions
2,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,00,gkg
3,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,15,export
4,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,15,mentions
...,...,...,...,...,...,...,...
195,http://data.gdeltproject.org/gdeltv2/201502191...,2015,02,19,17,15,export
196,http://data.gdeltproject.org/gdeltv2/201502191...,2015,02,19,17,15,mentions
197,http://data.gdeltproject.org/gdeltv2/201502191...,2015,02,19,17,15,gkg
198,http://data.gdeltproject.org/gdeltv2/201502191...,2015,02,19,17,30,export


In [27]:
p = 0

for i in liste:

    try:
        url = urlopen(i)
        k = i.split("/")[-1].split(".zip")[0]
        zipfile = ZipFile(BytesIO(url.read()))
        FFdata = pd.read_csv(zipfile.open(k), header=None,on_bad_lines='skip', sep="\t", engine='python')
        print(p, i)
        pass

    except:
        print(p, i, "problème")
        pass
        
    p+=1

0 http://data.gdeltproject.org/gdeltv2/20150218230000.export.CSV.zip
1 http://data.gdeltproject.org/gdeltv2/20150218230000.mentions.CSV.zip
2 http://data.gdeltproject.org/gdeltv2/20150218230000.gkg.csv.zip
3 http://data.gdeltproject.org/gdeltv2/20150218231500.export.CSV.zip
4 http://data.gdeltproject.org/gdeltv2/20150218231500.mentions.CSV.zip
5 http://data.gdeltproject.org/gdeltv2/20150218231500.gkg.csv.zip
6 http://data.gdeltproject.org/gdeltv2/20150218233000.export.CSV.zip
7 http://data.gdeltproject.org/gdeltv2/20150218233000.mentions.CSV.zip
8 http://data.gdeltproject.org/gdeltv2/20150218233000.gkg.csv.zip
9 http://data.gdeltproject.org/gdeltv2/20150218234500.export.CSV.zip
10 http://data.gdeltproject.org/gdeltv2/20150218234500.mentions.CSV.zip
11 http://data.gdeltproject.org/gdeltv2/20150218234500.gkg.csv.zip
12 http://data.gdeltproject.org/gdeltv2/20150219000000.export.CSV.zip
13 http://data.gdeltproject.org/gdeltv2/20150219000000.mentions.CSV.zip
14 http://data.gdeltproject.org/

118 http://data.gdeltproject.org/gdeltv2/20150219104500.mentions.CSV.zip
119 http://data.gdeltproject.org/gdeltv2/20150219104500.gkg.csv.zip
120 http://data.gdeltproject.org/gdeltv2/20150219110000.export.CSV.zip
121 http://data.gdeltproject.org/gdeltv2/20150219110000.mentions.CSV.zip
122 http://data.gdeltproject.org/gdeltv2/20150219110000.gkg.csv.zip
123 http://data.gdeltproject.org/gdeltv2/20150219111500.export.CSV.zip
124 http://data.gdeltproject.org/gdeltv2/20150219111500.mentions.CSV.zip
125 http://data.gdeltproject.org/gdeltv2/20150219111500.gkg.csv.zip
126 http://data.gdeltproject.org/gdeltv2/20150219113000.export.CSV.zip
127 http://data.gdeltproject.org/gdeltv2/20150219113000.mentions.CSV.zip
128 http://data.gdeltproject.org/gdeltv2/20150219113000.gkg.csv.zip
129 http://data.gdeltproject.org/gdeltv2/20150219114500.export.CSV.zip
130 http://data.gdeltproject.org/gdeltv2/20150219114500.mentions.CSV.zip
131 http://data.gdeltproject.org/gdeltv2/20150219114500.gkg.csv.zip
132 http://

In [28]:
def verify_url(u):
    try:
        url = urlopen(u)
        k = u.split("/")[-1].split(".zip")[0]
        zipfile = ZipFile(BytesIO(url.read()))
        FFdata = pd.read_csv(zipfile.open(k), header=None,on_bad_lines='skip', sep="\t", engine='python')
        return True
        pass

    except:
        return False
        pass

In [29]:
df['work'] = df['url'].apply(lambda x : verify_url(x)) ; df

,url,annee,mois,jour,heur,minute,type_csv,work
0,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,00,export,True
1,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,00,mentions,True
2,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,00,gkg,True
3,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,15,export,True
4,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,15,mentions,True
...,...,...,...,...,...,...,...,...
195,http://data.gdeltproject.org/gdeltv2/201502191...,2015,02,19,17,15,export,True
196,http://data.gdeltproject.org/gdeltv2/201502191...,2015,02,19,17,15,mentions,True
197,http://data.gdeltproject.org/gdeltv2/201502191...,2015,02,19,17,15,gkg,True
198,http://data.gdeltproject.org/gdeltv2/201502191...,2015,02,19,17,30,export,True


In [30]:
for i in range(df.shape[0]):
    if df.iloc[i,6] == "gkg" and df.iloc[i,7] == False:
        df.iloc[i+1,7] = False
        df.iloc[i+2,7] = False
        
    elif df.iloc[i,6] == "export" and df.iloc[i,7] == False:
        df.iloc[i-1,7] = False
        df.iloc[i+1,7] = False
    
    elif df.iloc[i,6] == "mentions" and df.iloc[i,7] == False:
        df.iloc[i-2,7] = False
        df.iloc[i-1,7] = False
        

In [31]:
df

,url,annee,mois,jour,heur,minute,type_csv,work
0,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,00,export,True
1,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,00,mentions,True
2,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,00,gkg,True
3,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,15,export,True
4,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,15,mentions,True
...,...,...,...,...,...,...,...,...
195,http://data.gdeltproject.org/gdeltv2/201502191...,2015,02,19,17,15,export,True
196,http://data.gdeltproject.org/gdeltv2/201502191...,2015,02,19,17,15,mentions,True
197,http://data.gdeltproject.org/gdeltv2/201502191...,2015,02,19,17,15,gkg,True
198,http://data.gdeltproject.org/gdeltv2/201502191...,2015,02,19,17,30,export,True


In [32]:
df = df.loc[df['work']!=False]
df

,url,annee,mois,jour,heur,minute,type_csv,work
0,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,00,export,True
1,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,00,mentions,True
2,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,00,gkg,True
3,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,15,export,True
4,http://data.gdeltproject.org/gdeltv2/201502182...,2015,02,18,23,15,mentions,True
...,...,...,...,...,...,...,...,...
195,http://data.gdeltproject.org/gdeltv2/201502191...,2015,02,19,17,15,export,True
196,http://data.gdeltproject.org/gdeltv2/201502191...,2015,02,19,17,15,mentions,True
197,http://data.gdeltproject.org/gdeltv2/201502191...,2015,02,19,17,15,gkg,True
198,http://data.gdeltproject.org/gdeltv2/201502191...,2015,02,19,17,30,export,True


In [34]:
df['url'].tolist()

['http://data.gdeltproject.org/gdeltv2/20150218230000.export.CSV.zip',
 'http://data.gdeltproject.org/gdeltv2/20150218230000.mentions.CSV.zip',
 'http://data.gdeltproject.org/gdeltv2/20150218230000.gkg.csv.zip',
 'http://data.gdeltproject.org/gdeltv2/20150218231500.export.CSV.zip',
 'http://data.gdeltproject.org/gdeltv2/20150218231500.mentions.CSV.zip',
 'http://data.gdeltproject.org/gdeltv2/20150218231500.gkg.csv.zip',
 'http://data.gdeltproject.org/gdeltv2/20150218233000.export.CSV.zip',
 'http://data.gdeltproject.org/gdeltv2/20150218233000.mentions.CSV.zip',
 'http://data.gdeltproject.org/gdeltv2/20150218233000.gkg.csv.zip',
 'http://data.gdeltproject.org/gdeltv2/20150218234500.export.CSV.zip',
 'http://data.gdeltproject.org/gdeltv2/20150218234500.mentions.CSV.zip',
 'http://data.gdeltproject.org/gdeltv2/20150218234500.gkg.csv.zip',
 'http://data.gdeltproject.org/gdeltv2/20150219000000.export.CSV.zip',
 'http://data.gdeltproject.org/gdeltv2/20150219000000.mentions.CSV.zip',
 'http:/

In [35]:
liste_de_dataframe = list()

liste = df['url'].tolist()

for i in liste:
    url = urlopen(i)

    k = i.split("/")[-1].split(".zip")[0]
    
    zipfile = ZipFile(BytesIO(url.read()))
    FFdata = pd.read_csv(zipfile.open(k), header=None,on_bad_lines='skip', sep="\t")
    liste_de_dataframe.append(FFdata)

liste_de_dataframe

[             0         1       2     3          4    5        6    7    8   \
 0     410412347  20140218  201402  2014  2014.1315  NaN      NaN  NaN  NaN   
 1     410412348  20140218  201402  2014  2014.1315  NaN      NaN  NaN  NaN   
 2     410412349  20140218  201402  2014  2014.1315  NaN      NaN  NaN  NaN   
 3     410412350  20140218  201402  2014  2014.1315  NaN      NaN  NaN  NaN   
 4     410412351  20140218  201402  2014  2014.1315  NaN      NaN  NaN  NaN   
 ...         ...       ...     ...   ...        ...  ...      ...  ...  ...   
 2266  410414613  20150218  201502  2015  2015.1315  ran     RANA  NaN  NaN   
 2267  410414614  20150218  201502  2015  2015.1315  ran     RANA  NaN  NaN   
 2268  410414615  20150218  201502  2015  2015.1315  ran     RANA  NaN  NaN   
 2269  410414616  20150218  201502  2015  2015.1315  ran     RANA  NaN  NaN   
 2270  410414617  20150218  201502  2015  2015.1315  tib  TIBETAN  NaN  NaN   
 
        9    10   11   12   13   14   15          

# -------------------------------------------------

# -------------------------------------------------

# -------------------------------------------------

# Jointure

In [3]:
mt_MentionDocTranslationInfo = pd.DataFrame(mentions_translation[['GlobalEventID','MentionDocTranslationInfo']])
mt_MentionDocTranslationInfo.rename(columns={'MentionDocTranslationInfo':'mt_MentionDocTranslationInfo'}, inplace=True)

In [4]:
df = mentions.merge(mt_MentionDocTranslationInfo, on='GlobalEventID', how='left')

In [5]:
dk = df.merge(export, on='GlobalEventID', how='left')

In [6]:
dk

,GlobalEventID,EventTimeDate,MentionTimeDate,MentionType,MentionSourceName,MentionIdentifier,SentenceID,Actor1CharOffset,Actor2CharOffset,ActionCharOffset,InRawText,Confidence,MentionDocLen,MentionDocTone,MentionDocTranslationInfo,Extras,mt_MentionDocTranslationInfo,Day,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,Actor1Religion2Code,Actor1Type1Code,Actor1Type2Code,Actor1Type3Code,Actor2Code,Actor2Name,Actor2CountryCode,Actor2KnownGroupCode,Actor2EthnicCode,Actor2Religion1Code,Actor2Religion2Code,Actor2Type1Code,Actor1Type2Code,Actor1Type3Code,IsRootEvent,EventCode,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,Actor1Geo_Type,Actor1Geo_Fullname,Actor1Geo_CountryCode,Actor1Geo_ADM1Code,Actor1Geo_ADM2Code,Actor1Geo_Lat,Actor1Geo_Long,Actor1Geo_FeatureID,Actor2Geo_Type,Actor2Geo_Fullname,Actor2Geo_CountryCode,Actor2Geo_ADM1Code,Actor2Geo_ADM2Code,Actor2Geo_Lat,Actor2Geo_Long,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_Fullname,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,410412347,20150218230000,20150218230000,1,dailymaverick.co.za,http://www.dailymaverick.co.za/article/2015-02...,19,-1,4594,4634,1,50,6665,-4.477612,NaN,NaN,NaN,20140218.0,201402.0,2014.0,2014.1315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GOV,PRESIDENT,NaN,NaN,NaN,NaN,NaN,GOV,NaN,NaN,0.0,43.0,43.0,4.0,1.0,2.8,5.0,1.0,5.0,-4.477612,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,"Waterkloof, Free State, South Africa",SF,SF03,77359,-30.3098,25.2971,-1299321,4.0,"Waterkloof, Free State, South Africa",SF,SF03,77359,-30.3098,25.2971,-1299321,2.015022e+13,http://www.dailymaverick.co.za/article/2015-02...
1,410412348,20150218230000,20150218230000,1,indiatimes.com,http://timesofindia.indiatimes.com/city/bengal...,2,-1,300,344,1,50,2541,2.078522,NaN,NaN,NaN,20140218.0,201402.0,2014.0,2014.1315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IND,MUMBAI,IND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,42.0,42.0,4.0,1.0,1.9,5.0,1.0,5.0,2.078522,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,"Mumbai, Maharashtra, India",IN,IN16,70184,18.9750,72.8258,-2092174,4.0,"Bengaluru, Karnataka, India",IN,IN19,70159,12.9833,77.5833,-2090174,2.015022e+13,http://timesofindia.indiatimes.com/city/bengal...
2,410412349,20150218230000,20150218230000,1,voxy.co.nz,http://www.voxy.co.nz/entertainment/coast-new-...,4,-1,1297,1232,0,10,2576,7.517084,NaN,NaN,NaN,20140218.0,201402.0,2014.0,2014.1315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NZL,KIWI,NZL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,42.0,42.0,4.0,1.0,1.9,1.0,1.0,1.0,7.517084,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,New Zealand,NZ,NZ,NaN,-41.0000,174.0000,NZ,4.0,"Great Southern, Victoria, Australia",AS,AS07,5387,-36.0667,146.4830,-1576477,2.015022e+13,http://www.voxy.co.nz/entertainment/coast-new-...
3,410412350,20150218230000,20150218230000,1,voxy.co.nz,http://www.voxy.co.nz/entertainment/coast-new-...,4,-1,1298,1233,1,20,2576,7.517084,NaN,NaN,NaN,20140218.0,201402.0,2014.0,2014.1315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NZL,KIWI,NZL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,42.0,42.0,4.0,1.0,1.9,2.0,1.0,2.0,7.517084,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,New Zealand,NZ,NZ,NaN,-41.0000,174.0000,NZ,1.0,New Zealand,NZ,NZ,NaN,-41.0000,174.0000,NZ,2.015022e+13,http://www.voxy.co.nz/entertainment/coast-new-...
4,410412351,20150218230000,20150218230000,1,eastidahonews.com,http://www.eastidahonews.com/2015/02/neil-patr...,1,-1,103,122,1,100,1432,0.000000,NaN,NaN,NaN,20140218.0,201402.0,2014.0,2014.1315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,LOS ANGELES,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,42.0,42.0,4.0,1.0,1.9,10.0,1.0,10.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,"Idaho, United States",US,USID,NaN,44.2394,-114.5100,ID,2.0,"Idaho, United States",US,USID,NaN,44.2394,-114.5100,ID,2.015022e+13,http://www.eastidahonews.com/2015/02/neil-patr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [7]:
export

,GlobalEventID,Day,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,Actor1Religion2Code,Actor1Type1Code,Actor1Type2Code,Actor1Type3Code,Actor2Code,Actor2Name,Actor2CountryCode,Actor2KnownGroupCode,Actor2EthnicCode,Actor2Religion1Code,Actor2Religion2Code,Actor2Type1Code,Actor1Type2Code,Actor1Type3Code,IsRootEvent,EventCode,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,Actor1Geo_Type,Actor1Geo_Fullname,Actor1Geo_CountryCode,Actor1Geo_ADM1Code,Actor1Geo_ADM2Code,Actor1Geo_Lat,Actor1Geo_Long,Actor1Geo_FeatureID,Actor2Geo_Type,Actor2Geo_Fullname,Actor2Geo_CountryCode,Actor2Geo_ADM1Code,Actor2Geo_ADM2Code,Actor2Geo_Lat,Actor2Geo_Long,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_Fullname,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,410412347,20140218,201402,2014,2014.1315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GOV,PRESIDENT,NaN,NaN,NaN,NaN,NaN,GOV,NaN,NaN,0,43,43,4,1,2.8,5,1,5,-4.477612,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Waterkloof, Free State, South Africa",SF,SF03,77359,-30.3098,25.2971,-1299321,4,"Waterkloof, Free State, South Africa",SF,SF03,77359,-30.3098,25.2971,-1299321,20150218230000,http://www.dailymaverick.co.za/article/2015-02...
1,410412348,20140218,201402,2014,2014.1315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IND,MUMBAI,IND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,42,42,4,1,1.9,5,1,5,2.078522,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Mumbai, Maharashtra, India",IN,IN16,70184,18.9750,72.8258,-2092174,4,"Bengaluru, Karnataka, India",IN,IN19,70159,12.9833,77.5833,-2090174,20150218230000,http://timesofindia.indiatimes.com/city/bengal...
2,410412349,20140218,201402,2014,2014.1315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NZL,KIWI,NZL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,42,42,4,1,1.9,1,1,1,7.517084,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,New Zealand,NZ,NZ,NaN,-41.0000,174.0000,NZ,4,"Great Southern, Victoria, Australia",AS,AS07,5387,-36.0667,146.4830,-1576477,20150218230000,http://www.voxy.co.nz/entertainment/coast-new-...
3,410412350,20140218,201402,2014,2014.1315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NZL,KIWI,NZL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,42,42,4,1,1.9,2,1,2,7.517084,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,New Zealand,NZ,NZ,NaN,-41.0000,174.0000,NZ,1,New Zealand,NZ,NZ,NaN,-41.0000,174.0000,NZ,20150218230000,http://www.voxy.co.nz/entertainment/coast-new-...
4,410412351,20140218,201402,2014,2014.1315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,LOS ANGELES,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,42,42,4,1,1.9,10,1,10,0.000000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,"Idaho, United States",US,USID,NaN,44.2394,-114.5100,ID,2,"Idaho, United States",US,USID,NaN,44.2394,-114.5100,ID,20150218230000,http://www.eastidahonews.com/2015/02/neil-patr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266,410414613,20150218,201502,2015,2015.1315,ran,RANA,NaN,NaN,ran,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,40,40,4,1,1.0,6,1,6,-3.703704,4,"Islamabad, Islamabad, Pakistan",PK,PK08,23674,33.7000,73.1667,-2762812,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Islamabad, Islamabad, Pakistan",PK,PK08,23674,33.7000,73.1667,-2762812,20150218230000,http://www.dailytimes.com.pk/national/19-Feb-2...
2267,410414614,20150218,201502,2015,2015.1315,ran,RANA,NaN,NaN,ran,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,46,46,4,1,7.0,6,1,6,-3.703704,4,"Agha Muhammad, Balochistan, Pakistan",PK,PK02,3606,30.5699,66.5961,356668,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Agha Muhammad, Balochistan, Pakistan",PK,PK02,3606,30.5699,66.5961,356668,20150218230000,http://www.dailytimes.com.pk/national/19-Feb-2...
2268,410414615,20150218,201502,2015,2015.1315,ran,RANA,NaN,NaN,ran,NaN,NaN,NaN,NaN,NaN,PAK,PAKI